In [17]:
# The neccessary packages
## In-house packages
from saveload import save,load
from ml_sample_generator import load_fire_detection, load_topography
## Pacakages from anywhere else
import os
from os import path as osp 
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import rowcol, xy
from pyproj import CRS, Transformer

In [6]:
# The file paths
tif = osp.join('C:/', 'Users', 'T-Spe', 'OneDrive', 'School', "Fall '25", "Master's Project", 'test')
local = osp.join('C:/', 'Users', 'T-Spe', 'Downloads')

file_paths = {"elevation_path": osp.join(tif ,'LH20_Elev_220.tif'),
            "slope_path": osp.join(tif, 'LH20_SlpP_220.tif'),
            "aspect_path": osp.join(tif, 'LH20_Asp_220.tif'),
            #"process_path": osp.join(local, 'processed_output.nc'),
            "fire_path": osp.join(local,'ml_data'),
            "nc_output_path": osp.join(local, 'row_col_mask.nc') 
                }


In [7]:
def create_netcdf_with_row_col_and_mask(
    lon_array,
    lat_array,
    raster_crs,
    transform,
    raster_shape,
    output_file,
    debug=False
):
    """
    Create a NetCDF file with row, column indices, and a spatial mask for provided lon/lat data.
    
    Args:
        lon_array (np.ndarray): Array of longitudes in WGS84.
        lat_array (np.ndarray): Array of latitudes in WGS84.
        raster_crs (str): CRS of the raster (e.g., "EPSG:5070").
        transform (Affine): Rasterio affine transform of the raster.
        raster_shape (tuple): Shape of the raster (rows, cols).
        output_file (str): Path to the output NetCDF file.
        debug (bool): Whether to enable debug messages.
    """
    print(f"Creating NetCDF file at {output_file}...")

    # Initialize transformer
    transformer = Transformer.from_crs("EPSG:4326", raster_crs, always_xy=True)

    # Reproject lon/lat to raster CRS
    print("Transforming coordinates to raster CRS...")
    raster_lon, raster_lat = transformer.transform(lon_array, lat_array)

    # Calculate row and column indices
    inv_transform = ~transform
    cols, rows = inv_transform * (raster_lon, raster_lat)

    # Round indices and convert to integers
    rows = np.round(rows).astype(int)
    cols = np.round(cols).astype(int)

    # Create a valid mask based on raster shape
    valid_mask = (
        (rows >= 0) & (rows < raster_shape[0]) &
        (cols >= 0) & (cols < raster_shape[1])
    )
    rows_valid = rows[valid_mask]
    cols_valid = cols[valid_mask]

    if debug:
        print(f"Valid row indices: {rows_valid}")
        print(f"Valid col indices: {cols_valid}")
        print(f"Valid mask shape: {valid_mask.shape}")
        print(f"Number of valid points: {np.sum(valid_mask)}")

    # Write to NetCDF
    with Dataset(output_file, 'w', format='NETCDF4') as nc_file:
        # Define dimensions
        nc_file.createDimension('points', len(rows_valid))
        nc_file.createDimension('mask', len(valid_mask))

        # Create variables
        rows_var = nc_file.createVariable('rows', 'i4', ('points',), zlib=True)
        cols_var = nc_file.createVariable('cols', 'i4', ('points',), zlib=True)
        mask_var = nc_file.createVariable('valid_mask', 'i1', ('mask',), zlib=True)

        # Write data
        rows_var[:] = rows_valid
        cols_var[:] = cols_valid
        mask_var[:] = valid_mask.astype(int)

    print(f"NetCDF file saved: {output_file}")

In [13]:
def validate_row_col(rows, cols, raster_crs, transform):
    """
    Validate the row and column indices by reprojecting them back to longitude and latitude.

    Args:
        rows (np.ndarray): Array of row indices.
        cols (np.ndarray): Array of column indices.
        raster_crs (str): CRS of the raster (e.g., "EPSG:5070").
        transform (Affine): Rasterio affine transform of the raster.

    Returns:
        tuple: Reprojected longitude and latitude arrays.
    """
    print("Validating row and column indices by reprojecting back to longitude and latitude...")

    # Use rasterio's transform to compute the coordinates
    raster_coords = xy(transform, rows, cols, offset='center')

    # Extract the projected coordinates
    raster_x, raster_y = raster_coords

     # Step 2: Reproject from the raster CRS to WGS84
    transformer = Transformer.from_crs(raster_crs, "EPSG:4326", always_xy=True)
    lon_wgs84, lat_wgs84 = transformer.transform(raster_x, raster_y)

    #print(f"First 10 Reprojected WGS84 Longitudes: {lon_wgs84[:10]}")
    #print(f"First 10 Reprojected WGS84 Latitudes: {lat_wgs84[:10]}")

    # Verify if any NaN or invalid values exist
    if np.isnan(lon_wgs84).any() or np.isnan(lat_wgs84).any():
        raise ValueError("Reprojected coordinates contain NaN values.")

    return lon_wgs84, lat_wgs84

In [8]:
# Hardcoding time bounds to avoid loading meteorology data
# Dataset times are confirmed as: 2011-01-02 00:00:00 to 2024-09-14 23:00:00
# Hardcoded time bounds
time_lb = pd.Timestamp("2011-01-02 00:00:00")
time_ub = pd.Timestamp("2024-09-14 23:00:00")

print(f"Using hardcoded time bounds: time_lb = {time_lb}, time_ub = {time_ub}")

# Load topography data
topography = load_topography(file_paths)
raster_crs =topography["crs"]
transform = topography["transform"]
raster_shape = topography["elevation"].shape

# Load and filter fire detection data
fire_detection_data = load_fire_detection(file_paths, time_lb, time_ub, confidence_threshold =70)
lon_array = fire_detection_data['lon']
lat_array = fire_detection_data['lat']
#dates_fire = fire_detection_data['dates_fire']
#labels = fire_detection_data['labels']

Using hardcoded time bounds: time_lb = 2011-01-02 00:00:00, time_ub = 2024-09-14 23:00:00
Loading topography data...
Trying to open C:/Users\T-Spe\OneDrive\School\Fall '25\Master's Project\test\LH20_Elev_220.tif as <open DatasetReader name='C:/Users\T-Spe\OneDrive\School\Fall '25\Master's Project\test\LH20_Elev_220.tif' mode='r'>
Loading fire detection data...
Total data points: 267414861
Number of 'Fire' labels: 43852
Number of 'Fire' labels with confidence < 70: 10474
Number of remaining data points: 265165164
Number of remaining 'Fire' labels: 33337
Loaded 267414861 data points, filtered down to 265165164 based on confidence and labels.


In [9]:
output_netcdf = file_paths["nc_output_path"]
debug = True

# Call the function to create the saved file with row, col indices, and mask for spatial points outside of raster
# NOTE: The saved file is there so this block does not need to be ran again.
create_netcdf_with_row_col_and_mask(
    lon_array,
    lat_array,
    raster_crs,
    transform,
    raster_shape,
    output_netcdf,
    debug
)

Creating NetCDF file at C:/Users\T-Spe\Downloads\row_col_mask.nc...
Transforming coordinates to raster CRS...
Valid row indices: [   1    0    8 ... 1471 1477 1460]
Valid col indices: [4528 4093 4165 ... 1149 1197 1174]
Valid mask shape: (265165164,)
Number of valid points: 229300941
NetCDF file saved: C:/Users\T-Spe\Downloads\row_col_mask.nc


In [15]:
# Read the saved test file, check the shapes and minor data exploration
row_col_data = Dataset(output_netcdf)

# Check shapes
mask_shape = row_col_data.variables['valid_mask'][:].shape
rows_shape = row_col_data.variables['rows'][:].shape
cols_shape = row_col_data.variables['cols'][:].shape

print(f"The shape of the mask array is: {mask_shape}")
print(f"The shape of the row array is: {rows_shape}")
print(f"The shape of the column array is: {cols_shape}")

# Check min, max, and NaN values for 'valid_mask'
valid_mask = row_col_data.variables['valid_mask'][:]
print(f"Mask min: {valid_mask.min()}, Mask max: {valid_mask.max()}")
print(f"Mask contains NaNs: {np.isnan(valid_mask).any()}")
print(f"Mask contains None: {np.any(valid_mask == None)}")  # NaNs should suffice, but adding for clarity

# Check min, max, and NaN values for 'rows'
rows = row_col_data.variables['rows'][:]
print(f"Rows min: {rows.min()}, Rows max: {rows.max()}")
print(f"Rows contains NaNs: {np.isnan(rows).any()}")
print(f"Rows contains None: {np.any(rows == None)}")

# Check min, max, and NaN values for 'cols'
cols = row_col_data.variables['cols'][:]
print(f"Columns min: {cols.min()}, Columns max: {cols.max()}")
print(f"Columns contain NaNs: {np.isnan(cols).any()}")
print(f"Columns contain None: {np.any(cols == None)}")

The shape of the mask array is: (265165164,)
The shape of the row array is: (229300941,)
The shape of the column array is: (229300941,)
Mask min: 0, Mask max: 1
Mask contains NaNs: False
Mask contains None: False
Rows min: 0, Rows max: 5257
Rows contains NaNs: False
Rows contains None: False
Columns min: 0, Columns max: 4609
Columns contain NaNs: False
Columns contain None: False


In [18]:
# Reproject the (row,col) back to (lon, lat) and compare to (lon, lat) used to obtain (row, col)
valid_mask = row_col_data.variables['valid_mask'][:].astype(bool)
lon_array_valid = lon_array[valid_mask]
lat_array_valid = lat_array[valid_mask]

lon_reproj_row , lat_reproj_col = validate_row_col(rows, cols, raster_crs, transform)

diff_lon = np.abs(lon_reproj_row - lon_array_valid)
diff_lat = np.abs(lat_reproj_col - lat_array_valid)
print(f"Max Longitude Difference: {np.max(diff_lon)}")
print(f"Max Latitude Difference: {np.max(diff_lat)}")

Validating row and column indices by reprojecting back to longitude and latitude...
Max Longitude Difference: 0.00028825367053286755
Max Latitude Difference: 0.00027203518735774423
